In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold

## Function for Tuning Blackbox Regression Models: `tune_regression_model`

In [2]:
def tune_regression_model(algorithm, X, Y, reg_method = "Dropout", M = 100, K = 10, c = None, criterion = "MSE"):
    """
    This function uses different regularization methods to train a predicted model that optimizes the specific criterion.
    @param algorithm(required): A learning algorithm that takes X, Y as inputs and outputs a model 
    @param X(required): The matrix of indepdent/predictor variables
    @param Y(required): The outcome variable
    @param reg_method(default value is 'Dropout'): Regularization methods Can be 'Dropout', 'NoiseAddition' or 'Robust'
    @param M(default value is 100): The number of Monte Carlo replicates
    @param K(default value is 5): Number of CV folds for tuning hyperparameter
    @param c(only used when the method is 'Robust'): A 1xp matrix, means the column bounds for X or the bound for each predictor variable.
    The best hyperparameter here is a p-dimensional vector.
    @param criterion(default value is 'MSE'): A criterion to be used to evaluate the method.Can be either mean squared error (MSE) or mean absolute deviation (MAD)
    @return: A predictive model that optimizes the specific criterion
    """
    def dropout(algorithm, X, Y, phi):
        Z = np.random.binomial(1, phi, size=X.shape)
        return algorithm(X * Z/(1-phi) ,Y)
    
    def noiseAddition(algorithm, X, Y, lam):
        Z = np.random.normal(0, lam, size=X.shape)
        return algorithm(X + Z,Y)
    
    def robust(algorithm, X, Y, c, scoring=mean_squared_error):
        max_error=0
        D = np.zeros(X.shape) 
        for _ in range(100):
            delta_matrix = np.zeros((X.shape)) 
            for j in range(len(c)):
                # Randomly sample a value k such that k <= c_j
                k = np.random.uniform(low=0, high=c[j])
                # Generate a vector from a normal distribution
                delta = np.random.normal(0, 1, len(X))
                # Normalize the vector so the sum of squares equals k
                delta = delta * k / np.linalg.norm(delta)
                # Assign the vector to the j-th column of delta_matrix
                delta_matrix[:, j] = delta
            X_new = X + delta_matrix
            pred = algorithm(X_new, Y).predict(X_new)
            delta_error = scoring(Y, pred)
            if delta_error>max_error:
                max_error=delta_error
                D = delta_matrix

        return algorithm(X+D, Y)

    if criterion == 'MSE':
        scoring = mean_squared_error
    elif criterion == 'MAD':
        scoring = mean_absolute_error
    else:
        raise ValueError("Unsupported criterion")  
    
    if reg_method == "Robust":
        if not c:
            raise ValueError('Please input a vector c of column bounds to be used if the method specified is Robust')
        hp_list = [[np.random.uniform(low=0, high=cj) for cj in c] for i in range(10)]
        method = robust
    elif reg_method == "NoiseAddition":
        method = noiseAddition
        hp_list = np.linspace(0, 1, 10)
    elif reg_method == "Dropout":
        method = dropout
        hp_list = np.arange(0, 1, 0.1)
    else:
        raise ValueError('Unsupported regularization method')
        
    cv_error = []
    
    for hp in hp_list:
        error = []
        for train_index, test_index in KFold(n_splits = K).split(X):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            if reg_method != 'Robust':
                X_train = np.repeat(X_train, M, axis=0)
                Y_train = np.repeat(Y_train, M, axis=0)
            model = method(algorithm, X_train, Y_train, hp)
            pred = model.predict(X_test)
            error.append(scoring(Y_test, pred))
        
        cv_error.append(np.mean(error))
        
    hp_opt = hp_list[np.argmin(cv_error)]
    
    return method(algorithm, X, Y, hp_opt)

## Example

In [3]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
sample_size = 100
indices = np.random.choice(len(data.data), sample_size, replace=False)

X = data.data[indices]
Y = data.target[indices]

### Linear Algorithm

In [4]:
from sklearn.linear_model import LinearRegression
def algorithm(X, Y):
    return LinearRegression().fit(X, Y)

In [5]:
mod1_mse = tune_regression_model(algorithm, X, Y, reg_method = "Dropout", criterion = "MSE")
mod1_mad = tune_regression_model(algorithm, X, Y, reg_method = "Dropout", criterion = "MAD")

In [6]:
print(mean_squared_error(Y, mod1_mse.predict(X)))
print(mean_absolute_error(Y, mod1_mad.predict(X)))

1.029672579251916
0.7656641188593579


In [7]:
mod2_mse = tune_regression_model(algorithm, X, Y, reg_method = "NoiseAddition", criterion = "MSE")
mod2_mad = tune_regression_model(algorithm, X, Y, reg_method = "NoiseAddition", criterion = "MAD")

In [8]:
print(mean_squared_error(Y, mod2_mse.predict(X)))
print(mean_absolute_error(Y, mod2_mad.predict(X)))

0.5963324577740188
0.5265874605268295


In [9]:
mod3_mse = tune_regression_model(algorithm, X, Y, reg_method = "Robust", c = [1]*X.shape[1], criterion = "MSE")
mod3_mad = tune_regression_model(algorithm, X, Y, reg_method = "Robust", c = [1]*X.shape[1], criterion = "MAD")

In [10]:
print(mean_squared_error(Y, mod3_mse.predict(X)))
print(mean_absolute_error(Y, mod3_mad.predict(X)))

0.5648671642964664
0.5086161957582362


### Machine Learning Algorithm (Non-linear)

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
def algorithm(X, Y):
    return GradientBoostingRegressor().fit(X, Y)

In [12]:
mod1_mse = tune_regression_model(algorithm, X, Y, reg_method = "Dropout", criterion = "MSE")
mod1_mad = tune_regression_model(algorithm, X, Y, reg_method = "Dropout", criterion = "MAD")

In [13]:
print(mean_squared_error(Y, mod1_mse.predict(X)))
print(mean_absolute_error(Y, mod1_mad.predict(X)))

1.046605526969794
0.791171378286101


In [14]:
mod2_mse = tune_regression_model(algorithm, X, Y, reg_method = "NoiseAddition", criterion = "MSE")
mod2_mad = tune_regression_model(algorithm, X, Y, reg_method = "NoiseAddition", criterion = "MAD")

In [15]:
print(mean_squared_error(Y, mod2_mse.predict(X)))
print(mean_absolute_error(Y, mod2_mad.predict(X)))

0.6030703342255406
0.5360693028793765


In [16]:
mod3_mse = tune_regression_model(algorithm, X, Y, reg_method = "Robust", c = [1]*X.shape[1], criterion = "MSE")
mod3_mad = tune_regression_model(algorithm, X, Y, reg_method = "Robust", c = [1]*X.shape[1], criterion = "MAD")

In [17]:
print(mean_squared_error(Y, mod3_mse.predict(X)))
print(mean_absolute_error(Y, mod3_mad.predict(X)))

0.5653796778461474
0.5050878591756583
